# Parameter management

PyBamm comes with a set of pre-defined parameters for common chemistries. These files are located inside your PyBaMM installation directory, typically something like
```
../venv/lib/pythonX.Y/site-packages/pybamm/input/parameters
```

The `input/parameters` directory is organised as follows:
```
input/parameters
  lithium-ion/ # chemistry
    anodes/ # Component
      graphite_Chen2020/ # parameter set
        parameters.csv
	    graphite_LGM50_diffusivity_Chen2020.py
        ...
      graphite_mcmb2528_Marquis2019/
      graphite_Ecker2015/
      ...
    cathodes/
    cells/
    electrolytes/
    seis/
    separators/
    experiments/
  lead-acid/
    ...
```

When setting parameter values using `pybamm.ParameterValues` and a filename, the file is searched in several locations, described by
the variable `pybamm.PARAMETER_PATH`.
The default is 

In [1]:
import pybamm
pybamm.PARAMETER_PATH

['/home/tlestang/projects/PyBaMM/examples/notebooks',
 '/home/tlestang/virtualenvs/pybamm/lib/python3.7/site-packages/pybamm/input/parameters']

which means that parameter files will first be searched in the current directory, and then the `input/parameters` directory within the 
PyBaMM installation directory.

## Editing the default parameters
It is often much easier to define new parameters by editing exiting ones.

We recommend not to alter the default parameters provided with the PyBaMM installation. Instead, you can pull all parameter set for
a given chemistry into the current directory:

In [2]:
%%bash
if [[ ! -d "lithium-ion" ]]; then
echo "Directory lithium-ion does not exist."
fi

Directory lithium-ion does not exist.


In [3]:
%%bash
pybamm_edit_parameter lithium-ion

The above commands will create a `lithium-ion` directory in the current directory, populated with copies of the default parameters, for editing:

In [4]:
%%bash
ls lithium-ion

anodes
cathodes
cells
electrolytes
experiments
seis
separators


As an example, let's create a new parameter file for the `cell` component, for the `lithium-ion` chemistry. We first create a new directory `my_new_param_set`:

In [5]:
%%bash
mkdir lithium-ion/cells/my_new_param_set

and write some data in a file `my_new_param_set/param_file.csv`. This is done using python in this simple example, but it can be done using the text editor of your choice to create a new file or edit an existing file.

In [6]:
# Create example parameter file
f = open("lithium-ion/cells/my_new_param_set/param_file.csv", "w+")
f.write(
"""
Name [units],Value
a, 4
b, 5
c, 6
"""
)
f.close()

## Adding a parameter directory
You can add a search location by modifying the `PARAMETER_PATH` list.
Let's add our new parameter directory to the list, in first position so that it is searched first:

In [7]:
pybamm.PARAMETER_PATH.insert(0, "lithium-ion/cells/my_new_param_set")
pybamm.PARAMETER_PATH

['lithium-ion/cells/my_new_param_set',
 '/home/tlestang/projects/PyBaMM/examples/notebooks',
 '/home/tlestang/virtualenvs/pybamm/lib/python3.7/site-packages/pybamm/input/parameters']

Note that relative paths added to `pybamm.PARAMETER_PATH` are considered relative to the current directory. In the above example, if you change directory, the new parameter `my_new_param_set` will not be found.

Let's check that our new parameter file can be read:

In [8]:
pybamm.ParameterValues("param_file.csv")

{'a': 4,
 'b': 5,
 'c': 6}

## Contributing a new parameter set
Once you're happy with a particular parameter set of your creation, you may want to contribute it to the PyBaMM parameter distribution, so
that it comes bundled with future release of PyBaMM.
This will require you to open a Pull Request on the PyBaMM repository, a process that is described here.

A prerequisite is that you add you parameter set to the default parameter directory. You could manually copy the corresponding directory to your installation directory, but the command `pybamm_add_param` can do it for you:

In [9]:
%%bash
# pybamm_add_parameter <dir> <chemistry> <component>
pybamm_add_parameter lithium-ion/cells/my_new_param_set lithium-ion cells

Let's remove the `my_new_param_set` from the parameter search path to check that the parameter directory was added correctly to set of default parameters:

In [10]:
pybamm.PARAMETER_PATH.remove("lithium-ion/cells/my_new_param_set")
pybamm.PARAMETER_PATH

['/home/tlestang/projects/PyBaMM/examples/notebooks',
 '/home/tlestang/virtualenvs/pybamm/lib/python3.7/site-packages/pybamm/input/parameters']

In [11]:
pybamm.ParameterValues("lithium-ion/cells/my_new_param_set/param_file.csv")

{'a': 4,
 'b': 5,
 'c': 6}

If you think you made a mistake, you can always delete a specific parameter set using `pybamm_rm_param`.


In [12]:
%%bash
pybamm_rm_parameter -f lithium-ion/cells/my_new_param_set lithium-ion cells

Let's remove the local lithium-ion directory to leave this directory as we found it

In [13]:
%%bash
rm -rf lithium-ion